<a href="https://colab.research.google.com/github/SonnyDev/food_csp/blob/main/Food_CSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

# Data Preparation
FOODS = ["icecream", "banana", "chocolatecake", "lasagna", "steak", "rice", "chips", "broccoli", "beans"]
FEATURES = ["energy", "protein", "salt", "fat", "cost"]
dd_values = {
    "icecream": [1200, 50, 10, 120, 400],
    "banana": [800, 120, 5, 20, 120],
    "chocolatecake": [2500, 400, 20, 100, 600],
    "lasagna": [3000, 200, 100, 250, 450],
    "steak": [1800, 800, 50, 100, 1200],
    "rice": [1200, 50, 5, 20, 100],
    "chips": [2000, 50, 200, 200, 250],
    "broccoli": [700, 100, 10, 10, 125],
    "beans": [1900, 250, 60, 90, 150]
}
categories = {
    "desserts": ["icecream", "banana", "chocolatecake"],
    "mains": ["lasagna", "steak", "rice"],
    "sides": ["chips", "broccoli", "beans"]
}

# Decision Variables
food_selection = {}
for category, items in categories.items():
    for food in items:
        food_selection[(category, food)] = model.NewBoolVar(f'select_{category}_{food}')

# Constraints for selecting exactly one food per category
for category in categories.keys():
    model.Add(sum(food_selection[(category, food)] for food in categories[category]) == 1)

# Nutritional and Cost Constraints
total_energy = model.NewIntVar(0, 10000, 'total_energy')
total_protein = model.NewIntVar(0, 10000, 'total_protein')
total_salt = model.NewIntVar(0, 10000, 'total_salt')
total_fat = model.NewIntVar(0, 10000, 'total_fat')
total_cost = model.NewIntVar(0, 10000, 'total_cost')

# Linking food selection to nutritional values and cost
model.Add(total_energy == sum(dd_values[food][0] * food_selection[(category, food)] for category, foods in categories.items() for food in foods))
model.Add(total_protein == sum(dd_values[food][1] * food_selection[(category, food)] for category, foods in categories.items() for food in foods))
model.Add(total_salt == sum(dd_values[food][2] * food_selection[(category, food)] for category, foods in categories.items() for food in foods))
model.Add(total_fat == sum(dd_values[food][3] * food_selection[(category, food)] for category, foods in categories.items() for food in foods))
model.Add(total_cost == sum(dd_values[food][4] * food_selection[(category, food)] for category, foods in categories.items() for food in foods))

# Nutritional Goals and Limits
min_energy = 3300
min_protein = 500
max_salt = 180
max_fat = 320

model.Add(total_energy >= min_energy)
model.Add(total_protein >= min_protein)
model.Add(total_salt <= max_salt)
model.Add(total_fat <= max_fat)

# Objective: Minimize Total Cost
model.Minimize(total_cost)

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print("Optimal solution found with total cost:", solver.Value(total_cost))
    for category, foods in categories.items():
        for food in foods:
            if solver.Value(food_selection[(category, food)]) == 1:
                print(f"Selected {category}: {food} (Energy: {dd_values[food][0]}, Protein: {dd_values[food][1]}, Salt: {dd_values[food][2]}, Fat: {dd_values[food][3]}, Cost: {dd_values[food][4]})")
else:
    print("No solution found.")

Optimal solution found with total cost: 825
Selected desserts: chocolatecake (Energy: 2500, Protein: 400, Salt: 20, Fat: 100, Cost: 600)
Selected mains: rice (Energy: 1200, Protein: 50, Salt: 5, Fat: 20, Cost: 100)
Selected sides: broccoli (Energy: 700, Protein: 100, Salt: 10, Fat: 10, Cost: 125)
